# Introduction to Computer Vision - Exercise
---
We will once more use DALL-E generated images, starting with the one we already saw in the lecture material. Instructions for the individual tasks are given in <span style="color:blue">blue</span> color.

---

## Overview
  - [Grayscale Conversion](#Grayscale-Conversion)
  - [Corner detection](#Corner-detection)
- [Object detection](#Object-detection)
  - [Comparing face detectors](#Comparing-face-detectors)
  - [Comparing upper body detectors](#Comparing-upper-body-detectors)
  - [Critical assessment of detectors](#Critical-assessment-of-detectors)

---

In [ ]:
# These imports let you use opencv
import cv2 #opencv itself
import numpy as np # matrix manipulations

#the following are to do with this interactive notebook code
%matplotlib inline 
from matplotlib import pyplot as plt # this lets you draw inline pictures in the notebooks
# default size of figures
plt.rcParams['figure.figsize'] = [6, 6]

# we will use imgstr, which makes it easy to use different images
imgstr = 'img/fabel.jpg'

### Grayscale Conversion

We have seen that while we can do the necessary color conversions by hand, there are also built-in OpenCV functions to achieve this. Let us now do both for grayscale.

<div style="color:blue">Let's get started with a little conversion task.</div>

1. <div style="color:blue">Read the input image and obtain the individual channels.</div>
1. <div style="color:blue">Manually create a grayscale image based on the channels.</div>
1. <div style="color:blue">Use the built-in OpenCV funtion to produce a second grayscale image.</div>
1. <div style="color:blue">Display the images side by side, saying which is which.</div>
1. <div style="color:blue">Optionally also display the difference map between the two grascale images.</div>

**Hints**: 
* Check the formula for color conversions [here](https://docs.opencv.org/4.9.0/de/d25/imgproc_color_conversions.html) for the manual part.
* If you do the optional part, investigate a bit further to get an explanation for what you see.

In [ ]:
# Your solution goes here: (if you need more code/markdown cells, feel free to insert them as you see fit)


### Corner detection
In the lecture material, we already tried the Harris corner detector.  Here, you should try out a different one and compare the two.

* <div style="color:blue">Use Shi-Tomasi corner detection to find corners in our input image. Compare the result to what the Harris detector finds. Which one do you prefer (and why)?</div>

**Hint:** You can find an example in the [OpenCV tutorials](https://docs.opencv.org/4.9.0/db/d27/tutorial_py_table_of_contents_feature2d.html). Be aware, however, that we want to show the corner points on a color image.

In [ ]:
# Your solution goes here:


*Your answer goes here:*




## Object detection

You will be working with two more images generated by DALL-E - one of people at a party and one of people standing by a lake in a park.

<div>
<img src='img/Fest_gemischt.jpg' alt="People at a party" width=400>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='img/Personen_im_Park.jpg' alt="People in a park by a lake" width=400>
</div>

Your task will be to try out different Haar cascades, which you will have to find first.

* <div style="color:blue">Find sources for Haar cascades to be used with opencv <b>and</b> provide the links for these.</div>

*Your links go here:*




### Comparing face detectors

Your task is to compare the default face detector to an alternative one and to interpret the results.
* <div style="color:blue">Start by loading the image of people at a party from which you should create a grayscale version and display both side by side</div>
* <div style="color:blue">Use the default face detector as well as an alternative one and compare the results both visually as well as through an assessment.</div>

In [ ]:
# Your solution goes here: (if you need more code/markdown cells, feel free to insert them as you see fit)


*Your assessment goes here:* 



### Comparing upper body detectors

Your task is to compare the default face detector to an alternative one and to interpret the results.
* <div style="color:blue">Start by loading the image of people at a lake in the park from which you should create a grayscale version and display both side by side</div>
* <div style="color:blue">Use the default upper body detector as well as an alternative one and compare the results both visually as well as through an assessment.</div>

**Hint:** it might take you some time to find an alternative Haar cascade for the upper body, but it might be well worth it (it might make sense to include "Modesto Castrillon-Santana" or "mcs" in your respective searches).

In [ ]:
# Your solution goes here: (if you need more code/markdown cells, feel free to insert them as you see fit)



*Your assessment goes here:* 



### Critical assessment of detectors

It is now time to reflect on the usefulness of the Haar cascade detectors
* <div style="color:blue">Critically assess the usefulness of the detectors based on the experiments in this notebook. Can you generalize? What might be a compounding factor here?</div>

*Your assessment goes here:*

